In [1]:
%load_ext autoreload
%autoreload 2

import pickle, os
import numpy as np
import cvxpy as cp
from cartpole import Cartpole

# Problem parameters

In [2]:
n = 4; m = 3

N = 11 # horizon
dh = 0.05
Q = 10.*np.eye(n)
R = np.diag([1., 0, 0])
l = 1.
mc = 1.
mp = 1.
g = 10.
kappa, nu = 100., 30. # contact parameters

dist = 0.5
x_max = np.array([dist, np.pi/8, 2, 1])
x_min = -x_max

delta_min = np.array([-x_max[0] + l*x_min[1] - dist, \
             x_min[0] - l*x_max[1] - dist])
ddelta_min = np.array([-x_max[2] + l*x_min[3], \
            x_min[2] - l*x_max[3]])

delta_max = np.array([-x_min[0] + l*x_max[1] - dist, \
              x_max[0] - l*x_min[1] - dist])
ddelta_max = np.array([-x_min[2] + l*x_max[3], \
              x_max[2] - l*x_min[3]])

uc_min = -2.
uc_max = -uc_min
sc_min = kappa*delta_min + nu*ddelta_min
sc_max = kappa*delta_max + nu*ddelta_max

A = np.zeros((n,n))
A[:int(n/2),int(n/2):] = np.eye(int(n/2))
A[2,1] = g*mp/mc
A[3,1] = g*(mc+mp)/(l*mc)
Ak = np.eye(n) + dh*A

B = np.zeros((n,m))
B[2,0] = 1/mc
B[3,:] = np.array([1/(l*mc), -1/(l*mp), 1/(l*mp)])
Bk = dh*B

prob_params = [N, Ak, Bk, Q, R, x_min, x_max,\
    uc_min, uc_max, sc_min, sc_max, \
    delta_min, delta_max, ddelta_min, ddelta_max, \
    dh, g, l, mc, mp, kappa, \
    nu, dist]

# Setup pickle file for default parameters

In [4]:
#setup filenames
relative_path = os.getcwd()
dataset_name = 'default'

if not os.path.isdir(relative_path+'/data/'+dataset_name):
    os.mkdir(relative_path+'/data/'+dataset_name)

train_fn = relative_path+'/data/'+dataset_name+'/train.p'
test_fn = relative_path+'/data/'+dataset_name+'/test.p'

#load default parameter values
n = 4; m = 3

#define all possible params that can be varied
all_params = ['N', 'Ak', 'Bk', 'Q', 'R', 'x_min', 'x_max', \
              'uc_min', 'uc_max', 'sc_min', 'sc_max', \
              'delta_min',  'delta_max', 'ddelta_min', 'ddelta_max', \
              'dh',  'g', 'l', 'mc', 'mp', \
              'kappa', 'nu', 'dist', 'x0', 'xg']

##TODO(pculbert): implement custom param variation.
#plan: split all_params into static & sampled params
#in Problem object, keep param dict with keys == all_params
#on constructing problem, make every param key map to either static value
#or a cvxpy.Parameter object.

### VARY ANY DESIRED PARAM VALUES HERE ###

param_dict = {'N': N, 'Ak': Ak, 'Bk' : Bk, 'Q' : Q, 'R' : R,\
              'x_min':  x_min, 'x_max' : x_max, 'uc_min' : uc_min, \
              'uc_max' : uc_max, 'sc_min' : sc_min, 'sc_max' : sc_max, \
              'delta_min' : delta_min,  'delta_max' :  delta_max,  \
              'ddelta_min' :  ddelta_min, 'ddelta_max' : ddelta_max, \
              'dh' : dh,  'g' : g, 'l' : l, 'mc' : mc, 'mp' : mp, \
              'kappa' : kappa, 'nu' :  nu, 'dist' : dist}

#specify which parameters to sample, & their distributions
sampled_params = ['x0', 'xg']
sample_dists = {'x0' : lambda :  0.5*(x_min+x_max) + (np.random.rand(n)-0.5)*(x_max-x_min),\
                'xg' : lambda : np.zeros(n)}

#specify dataset sizes
num_train = 90; num_test = 10
num_probs = num_train + num_test

#write out config
config_fn = relative_path+"/config/"+dataset_name+".p"
config = [dataset_name, prob_params, sampled_params]
outfile = open(config_fn,"wb")
pickle.dump(config,outfile); outfile.close()

In [134]:
#create cartpole object, w/ config file from above
prob = Cartpole()

#create numpy containers for data: (params, x, u, y, J*, solve_time)
params = {}
if 'x0' in sampled_params:
    params['x0'] = np.zeros((num_probs,n))
if 'xg' in sampled_params:
    params['xg'] = np.zeros((num_probs,n))

X = np.zeros((num_probs, n, N)); U = np.zeros((num_probs, m, N-1))
Y = np.zeros((num_probs,4, N-1)).astype(int)
costs = np.zeros(num_probs); solve_times = np.zeros(num_probs)

#big for loop, sampling problem data, solving MICP, & storing
ii = 0
while ii < num_probs:
    p_dict = {}
    if 'x0' in sampled_params:
        params['x0'][ii,:] = sample_dists['x0']()
        p_dict['x0'] = params['x0'][ii,:]
    if 'xg' in sampled_params:
        params['xg'][ii,:] = sample_dists['xg']()
        p_dict['xg'] = params['xg'][ii,:]

    prob_success = False
    try:
        prob_success, cost, solve_time, optvals = prob.solve_micp(p_dict, solver=cp.MOSEK)
    except (KeyboardInterrupt, SystemExit):
        raise
    except:
        print('mosek failed at '.format(ii))
    
    if prob_success:
        costs[ii] = cost; solve_times[ii] = solve_time
        X[ii,:,:], U[ii,:,:], Y[ii,:,:] = optvals
        ii += 1

In [135]:
#post-processing + write out
train_params = {}; test_params = {}
if 'x0' in sampled_params:
    train_params['x0'] = params['x0'][:num_train,:]
    test_params['x0'] = params['x0'][num_train:,:]
if 'xg' in sampled_params:
    train_params['xg'] = params['xg'][:num_train,:]
    test_params['xg'] = params['xg'][num_train:,:]
    
train_data = [train_params]
train_data += [X[:num_train,:,:], U[:num_train,:,:], Y[:num_train,:,:]]
train_data += [costs[:num_train], solve_times[:num_train]]

test_data = [test_params]
test_data += [X[num_train:,:,:], U[num_train:,:,:], Y[num_train:,:,:]]
test_data += [costs[num_train:], solve_times[num_train:]]

train_file = open(train_fn,'wb')
pickle.dump(train_data,train_file); train_file.close()

test_file = open(test_fn, 'wb')
pickle.dump(test_data,test_file); test_file.close()

In [139]:
train_file = open(train_fn,'rb')
p, x, u, y, c, s = pickle.load(train_file); train_file.close()